# Lesson 8: Multi-Agent Orchestration Patterns

This interactive notebook teaches you how to build systems with multiple collaborating agents:

- ✅ Graph pattern for deterministic workflows
- ✅ Swarm pattern for autonomous collaboration
- ✅ Workflow pattern for sequential coordination
- ✅ Agent specialization and role definition
- ✅ State sharing between agents
- ✅ Choosing the right multi-agent pattern

**Estimated time:** 5-6 hours

**What you'll build:** Research teams, autonomous swarms, and data processing pipelines!

## Setup

Import necessary modules and configure the environment:

In [ ]:
from lesson_utils import load_environment, create_working_model, check_api_keys
from strands import Agent
from strands.multiagent import GraphBuilder, Swarm
from strands_tools import workflow

# Load environment and check API keys
load_environment()
check_api_keys()

print("🎯 Lesson 8: Multi-Agent Orchestration Patterns")
print("=" * 50)

## Part 1: Graph Pattern - Deterministic Workflows

The **Graph pattern** enables deterministic multi-agent workflows with:
- **Explicit execution flow** - You define the path between agents
- **Conditional routing** - Add edges based on conditions
- **Parallel execution** - Multiple agents can run simultaneously
- **Converging paths** - Results from parallel branches can merge

### When to Use Graph Pattern:
- Customer support routing (intent-based branching)
- Data validation with error paths
- Document approval workflows
- Any scenario requiring predictable, controlled execution flow

### Example: Research Workflow

We'll create a research workflow where:
1. **Researcher** gathers information
2. **Analyst** and **Fact Checker** work in parallel on the research
3. **Report Writer** combines both outputs into a final report

In [ ]:
# Multi-agent workflows need higher max_tokens (1500 vs default 500)
model = create_working_model("multiagent")

if model:
    print("\n📊 Creating specialized research workflow agents...")

    # Create specialized agents for different research tasks
    researcher = Agent(
        model=model,
        name="researcher",
        system_prompt="""You are a research specialist. Your job is to gather
        key information on the given topic. Focus on finding factual,
        up-to-date information. Be concise but thorough."""
    )

    analyst = Agent(
        model=model,
        name="analyst",
        system_prompt="""You are a data analysis specialist. Analyze the research
        findings and extract key insights, patterns, and trends. Provide
        clear analysis with supporting evidence."""
    )

    fact_checker = Agent(
        model=model,
        name="fact_checker",
        system_prompt="""You are a fact-checking specialist. Verify the accuracy
        of information and identify any potential issues or inconsistencies.
        Flag anything that needs verification."""
    )

    report_writer = Agent(
        model=model,
        name="report_writer",
        system_prompt="""You are a report writing specialist. Create a clear,
        well-structured summary combining research, analysis, and fact-checking.
        Be concise and professional."""
    )

    print("✅ Created 4 specialized agents: researcher, analyst, fact_checker, report_writer\n")

    # Build the graph using GraphBuilder
    print("🔧 Building deterministic graph workflow...")
    builder = GraphBuilder()

    # Add nodes (agents) to the graph
    builder.add_node(researcher, "research")
    builder.add_node(analyst, "analysis")
    builder.add_node(fact_checker, "fact_check")
    builder.add_node(report_writer, "report")

    # Add edges (dependencies) - defines execution flow
    # Research must complete first, then analysis and fact-checking in parallel,
    # finally report combines both
    builder.add_edge("research", "analysis")
    builder.add_edge("research", "fact_check")
    builder.add_edge("analysis", "report")
    builder.add_edge("fact_check", "report")

    # Set entry point (where workflow starts)
    builder.set_entry_point("research")

    # Configure safety limits
    builder.set_execution_timeout(300)  # 5 minute timeout

    # Build the graph
    graph = builder.build()

    print("✅ Graph built successfully!")
    print("   Flow: research → (analysis + fact_check) → report\n")

    # Execute the graph
    print("🚀 Executing graph on task: 'Benefits of remote work'...")
    result = graph("What are 3 key benefits of remote work? Keep each benefit to 1-2 sentences.")

    print(f"\n📈 Graph execution completed!")
    print(f"   Status: {result.status}")
    print(f"   Nodes executed: {[node.node_id for node in result.execution_order]}")
    print(f"   Total nodes: {result.total_nodes}")
    print(f"   Completed: {result.completed_nodes}")

    # Show final report
    if "report" in result.results:
        final_result = result.results["report"].result
        print(f"\n📄 Final Report Preview:")
        print(f"   {str(final_result)[:200]}...")

    print("\n✅ Graph pattern demonstrates deterministic, structured workflows!")
else:
    print("⚠️ No API key available")

## Part 2: Swarm Pattern - Autonomous Collaboration

The **Swarm pattern** enables autonomous multi-agent collaboration with:
- **Agent-led handoffs** - Agents decide who to hand off to
- **Shared context** - All agents share conversation history
- **Emergent workflows** - Execution path emerges naturally
- **Dynamic routing** - No predefined paths, agents coordinate autonomously

### When to Use Swarm Pattern:
- Software development (research → design → code → review)
- Incident response (detection → diagnosis → resolution)
- Creative brainstorming with multiple viewpoints
- Any scenario benefiting from multiple specialized perspectives

### Example: GenAI Business Ideas Swarm

We'll create a simple swarm where:
1. **Idea Generator** creates GenAI business ideas
2. **Reviewer** evaluates and approves them

In [ ]:
# Multi-agent workflows need higher max_tokens (1500 vs default 500)
model = create_working_model("multiagent")

if model:
    print("\n🐝 Creating specialized swarm agents...")

    # Create specialized agents with clear roles
    # Focused on GenAI business ideas
    idea_generator = Agent(
        model=model,
        name="idea_generator",
        system_prompt="""You are a GenAI business consultant. Generate 2-3 specific 
        business ideas that leverage Generative AI technology. Focus on practical, 
        market-ready applications. For each idea, briefly mention: the problem solved, 
        target market, and AI capability used. Keep each idea to 2-3 sentences.
        Then hand off to reviewer.""",
        description="Generates GenAI business ideas"
    )

    reviewer = Agent(
        model=model,
        name="reviewer",
        system_prompt="""Review the GenAI business ideas for market viability and 
        technical feasibility. Provide brief feedback (1-2 sentences per idea). 
        If ideas are viable, approve and finish. Do NOT hand off again.""",
        description="Reviews and approves GenAI business ideas"
    )

    print("✅ Created 2 specialized agents: idea_generator, reviewer\n")

    # Create a swarm with these agents
    print("🔧 Creating autonomous swarm...")
    swarm = Swarm(
        [idea_generator, reviewer],  # Simple 2-agent swarm
        entry_point=idea_generator,  # Start with idea generation
        max_handoffs=5,
        max_iterations=10,
        execution_timeout=300.0,  # 5 minutes
        node_timeout=60.0  # 1 minute per agent
    )

    print("✅ Swarm created with autonomous handoff capabilities!\n")

    # Execute the swarm
    print("🚀 Executing swarm on task: 'GenAI business ideas for healthcare'...")
    result = swarm("Generate GenAI-powered business ideas for the healthcare sector. Focus on practical applications.")

    print(f"\n📈 Swarm execution completed!")
    print(f"   Status: {result.status}")
    print(f"   Agent handoffs: {[node.node_id for node in result.node_history]}")
    print(f"   Total iterations: {result.execution_count}")

    # Show results from each agent
    print(f"\n🔍 Results from agents:")
    for agent_name, node_result in result.results.items():
        agent_output = str(node_result.result)[:150]
        print(f"   {agent_name}: {agent_output}...")

    print("\n✅ Swarm pattern demonstrates autonomous, emergent collaboration!")
else:
    print("⚠️ No API key available")

## Part 3: Workflow Pattern - DAG-based Task Coordination

The **Workflow pattern** uses a Directed Acyclic Graph (DAG) to define tasks and their dependencies. The workflow tool from `strands-agents-tools`:
- **Automatically resolves dependencies** - Determines execution order based on DAG
- **Executes independent tasks in parallel** - Maximizes throughput
- **Passes context between dependent tasks** - Ensures information flow
- **Tracks workflow progress** - Status, pause, resume capabilities
- **Handles errors** - Automatic retries and recovery

This is fundamentally different from manually chaining agent calls!

### When to Use Workflow Pattern:
- Complex data processing pipelines with parallel stages
- Multi-step processes requiring dependency management
- Need for automatic parallel execution where possible
- Long-running processes needing pause/resume
- Production workflows requiring lifecycle management

### Example: Data Processing Pipeline with Parallel Execution

We'll create a workflow using the workflow tool:
1. **Data Collector** extracts raw data (no dependencies)
2. **Quality Checker** validates data (depends on collect)
3. **Data Analyzer** finds insights (depends on collect - runs in PARALLEL with quality check!)
4. **Report Generator** creates final report (depends on BOTH quality check AND analysis)

In [ ]:
# Multi-agent workflows need higher max_tokens (1500 vs default 500)
model = create_working_model("multiagent")

if model:
    print("\n⚙️  Creating agent with workflow tool...")

    # Create an agent with workflow capability
    orchestrator = Agent(
        model=model,
        name="workflow_orchestrator",
        tools=[workflow],
        system_prompt="You orchestrate multi-agent workflows."
    )

    print("✅ Created orchestrator agent with workflow tool\n")

    # Define workflow tasks with dependencies (DAG structure)
    print("🔧 Defining workflow DAG with dependencies...")
    print("   Structure: collect → (quality_check + analyze) → report")
    print("   Note: quality_check and analyze run in PARALLEL!\n")

    workflow_id = "data_pipeline"

    # Create the workflow
    print(f"📝 Creating workflow '{workflow_id}'...")
    create_result = orchestrator.tool.workflow(
        action="create",
        workflow_id=workflow_id,
        tasks=[
            {
                "task_id": "collect",
                "description": "Collect customer rating data (4.2/5 avg). Organize in structured format.",
                "system_prompt": "You collect and extract data from sources. Be concise.",
                "dependencies": [],  # No dependencies - starts immediately
                "priority": 5
            },
            {
                "task_id": "quality_check",
                "description": "Validate data quality and completeness. Check for issues.",
                "system_prompt": "You validate data quality. Check for missing values and inconsistencies. Be concise.",
                "dependencies": ["collect"],  # Waits for collect
                "priority": 3
            },
            {
                "task_id": "analyze",
                "description": "Analyze patterns and identify top 2 improvements from the data.",
                "system_prompt": "You analyze data to find patterns and insights. Use statistical thinking. Be concise.",
                "dependencies": ["collect"],  # Waits for collect (parallel with quality_check!)
                "priority": 3
            },
            {
                "task_id": "report",
                "description": "Generate final report combining quality check and analysis results.",
                "system_prompt": "You create clear, professional reports. Structure findings logically. Be concise.",
                "dependencies": ["quality_check", "analyze"],  # Waits for BOTH
                "priority": 1
            }
        ]
    )

    print(f"✅ Workflow created: {create_result['content']}\n")

    # Start the workflow
    print(f"🚀 Starting workflow execution...")
    start_result = orchestrator.tool.workflow(
        action="start",
        workflow_id=workflow_id
    )

    print(f"✅ Workflow started: {start_result['content']}\n")

    # Check workflow status
    print(f"📊 Checking workflow status...")
    status_result = orchestrator.tool.workflow(
        action="status",
        workflow_id=workflow_id
    )

    print(f"\n📈 Workflow Status:")
    print(f"{status_result['content']}\n")

    # Clean up
    print(f"🧹 Cleaning up workflow...")
    delete_result = orchestrator.tool.workflow(
        action="delete",
        workflow_id=workflow_id
    )
    print(f"✅ {delete_result['content']}\n")

    print("""
✅ Workflow tool demonstrates:
   • DAG-based task definition with dependencies
   • Automatic dependency resolution and execution order
   • Parallel execution of independent tasks (quality_check + analyze)
   • Automatic context passing between dependent tasks
   • Progress tracking with detailed status
   • Workflow lifecycle management (create, start, status, delete)

💡 Advanced features available:
   • Pause/Resume workflows
   • Automatic error recovery and retries
   • Dynamic resource management
   • Task-level metrics and monitoring
""")
else:
    print("⚠️ No API key available")

## Part 4: Pattern Comparison

### Understanding When to Use Each Pattern:

#### GRAPH PATTERN - Deterministic Workflows

**Use when:**
- You need conditional logic and branching
- Execution flow should be predictable
- Multiple paths need to converge (e.g., parallel analysis)
- You want explicit error handling paths

**Examples:**
- Customer support routing (intent-based branching)
- Data validation with error paths
- Document approval workflows

---

#### SWARM PATTERN - Autonomous Collaboration

**Use when:**
- Problem benefits from multiple specialized perspectives
- Path between agents should emerge naturally
- Agents need shared context and working memory
- You want autonomous agent-to-agent handoffs

**Examples:**
- Software development (research → design → code → review)
- Incident response (detection → diagnosis → resolution)
- Creative brainstorming with multiple viewpoints

---

#### WORKFLOW PATTERN - DAG-based Task Coordination

**Use when:**
- Complex data processing pipelines with parallel stages
- Multi-step processes requiring dependency management
- Need for automatic parallel execution where possible
- Long-running processes needing pause/resume
- Production workflows requiring lifecycle management

**Examples:**
- Data processing pipelines (extract → transform → load)
- Content creation workflows with parallel tasks
- Business processes with dependencies (ETL, reporting, onboarding)

---

### Quick Comparison Table:

| Feature | Graph | Swarm | Workflow Tool |
|---------|-------|-------|---------------|
| Implementation | GraphBuilder | Swarm class | workflow tool |
| Execution Flow | Controlled | Autonomous | DAG-based |
| Path Determination | Dev-defined | Agent-led | Dev-defined |
| Supports Cycles | Yes | Yes | No (DAG) |
| Parallel Execution | Yes | No | Yes |
| State Sharing | Shared dict | Shared ctx | Automatic |
| Complexity | Medium | Medium | Medium |
| Use Case Fit | Structured | Creative | Complex pipelines |
| Lifecycle Mgmt | No | No | Yes (pause/resume) |

### Key Insight:

Choose based on how much control and automation you need:
- **Graph** = Explicit control with dynamic decisions and parallel paths
- **Swarm** = Emergent path through autonomous agent handoffs
- **Workflow** = DAG-based pipelines with automatic dependency management

## Part 5: Shared State Across Patterns

### Shared State with invocation_state

Both Graph and Swarm patterns support passing shared state to all agents through the `invocation_state` parameter. This enables sharing context and configuration without exposing it to the LLM.

**Common use cases:**
- User identification (user_id, session_id)
- Configuration flags (debug_mode, feature_flags)
- Shared resources (database connections, API clients)
- Security context (authentication tokens, permissions)

In [ ]:
# Multi-agent workflows need higher max_tokens (1500 vs default 500)
model = create_working_model("multiagent")

if model:
    # Create simple agents
    agent1 = Agent(model=model, name="agent1", system_prompt="You are agent 1")
    agent2 = Agent(model=model, name="agent2", system_prompt="You are agent 2")

    # Create a simple graph
    builder = GraphBuilder()
    builder.add_node(agent1, "step1")
    builder.add_node(agent2, "step2")
    builder.add_edge("step1", "step2")
    builder.set_entry_point("step1")
    graph = builder.build()

    # Shared state that will be passed to all agents
    shared_state = {
        "user_id": "user_123",
        "session_id": "sess_456",
        "debug_mode": True,
        "feature_flags": {"new_ui": True, "beta_features": False}
    }

    print("\n🔧 Example: Passing shared state to Graph...\n")
    print(f"   Shared state: {shared_state}\n")

    # Execute with shared state (in real usage, tools would access via ToolContext)
    result = graph(
        "Process this request",
        **shared_state  # Passed as invocation_state
    )

    print(f"✅ Graph executed with shared state!")
    print(f"   Status: {result.status}")
    
    print("""
💡 Note: Tools can access invocation_state via ToolContext:

   @tool(context=True)
   def my_tool(query: str, tool_context: ToolContext) -> str:
       user_id = tool_context.invocation_state.get("user_id")
       debug = tool_context.invocation_state.get("debug_mode", False)
       # Use context for personalized, context-aware operations
       return f"Processing for user {user_id}"

This keeps sensitive data (like user IDs, API keys) out of conversation
history while still making it available to tools and hooks.
""")
else:
    print("⚠️ No API key available")

## Experiments

Now it's your turn! Try these experiments:

### Exercises:
1. **Customer Support Graph** - Build a graph with intent-based routing (billing → billing_agent, technical → tech_agent)
2. **Extended Swarm** - Create a 5+ agent swarm (add tester, documenter, security reviewer)
3. **Conditional Edges** - Add conditional edges to graph (quality check with retry loop)
4. **Error Handling** - Implement error handling path in graph workflow
5. **Pattern Comparison** - Compare same task with Graph vs Swarm (which works better?)
6. **Data Pipeline** - Build data validation workflow with multiple validation steps
7. **Nested Pattern** - Create nested pattern: Swarm node inside a Graph
8. **Shared State Tools** - Add tools that use invocation_state context (e.g., user preferences)
9. **Cyclic Graph** - Implement cyclic graph with max execution limits
10. **Code Generation System** - Build multi-agent system for code generation + review + testing

Use the cell below for your experiments:

In [ ]:
# Your experiments here!


## ✅ Success Criteria

You've completed Lesson 8 if:

- ✅ Graph pattern executes deterministic workflow with parallel branches
- ✅ Swarm pattern shows autonomous agent handoffs
- ✅ Workflow tool demonstrates DAG-based task coordination
- ✅ Workflow shows parallel execution (quality_check + analyze)
- ✅ State sharing works with invocation_state
- ✅ Understand when to use each pattern
- ✅ Can compare Graph vs Swarm vs Workflow Tool tradeoffs

## 💡 Key Concepts Learned

- **Graph Pattern** - Deterministic workflows with GraphBuilder, nodes, edges, conditional routing
- **Swarm Pattern** - Autonomous collaboration with handoff_to_agent, shared context
- **Workflow Pattern** - DAG-based task coordination with workflow tool from strands-agents-tools
- **Agent Specialization** - Creating role-specific agents with focused system prompts
- **State Sharing** - Using invocation_state for context across agents
- **Pattern Selection** - Choosing the right pattern based on control and complexity needs
- **Parallel Execution** - Graph and Workflow support parallel task execution
- **Lifecycle Management** - Workflow tool provides create/start/pause/resume/delete operations

## Next Steps

- **Lesson 9**: Distributed Agents - Agents-as-Tools and A2A protocol for cross-platform communication
- **Lesson 10**: Production - Safety, observability, and agent evaluation

Ready to continue? Open `lesson_09_distributed_agents.ipynb`!